In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('D://DS//credit.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [4]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [5]:
features = df.drop(['Class'],axis=1)
target = df['Class']

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [7]:
features_train,features_test,target_train,target_test = train_test_split(
features,target,train_size=0.7,random_state=12345)

In [8]:
log_class = LogisticRegression()
grid = {'C':10.0 **np.arange(-2,3),'penalty':['l1','l2']}
cv = KFold(n_splits=5,shuffle=False,random_state=None)

In [9]:
clf = GridSearchCV(log_class,grid,cv=cv,n_jobs=-1,scoring='f1_macro')
clf.fit(features_train,target_train)

C:\Users\rohit.pratapwar\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.81532363        nan 0.82734315        nan 0.8205935
        nan 0.8512813         nan 0.8312705 ]
  category=UserWarning


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [10]:
pred = clf.predict(features_test)

In [11]:
print(confusion_matrix(target_test,pred))
print(accuracy_score(target_test,pred))
print(classification_report(target_test,pred))

[[85260    38]
 [   46    99]]
0.9990168884519505
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85298
           1       0.72      0.68      0.70       145

    accuracy                           1.00     85443
   macro avg       0.86      0.84      0.85     85443
weighted avg       1.00      1.00      1.00     85443



In [12]:
class_weight = dict({0:1,1:100})

In [13]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(class_weight=class_weight)
classifier.fit(features_train,target_train)

RandomForestClassifier(class_weight={0: 1, 1: 100})

In [14]:
pred = classifier.predict(features_test)
print(confusion_matrix(target_test,pred))
print(accuracy_score(target_test,pred))
print(classification_report(target_test,pred))

[[85294     4]
 [   31   114]]
0.9995903701883126
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85298
           1       0.97      0.79      0.87       145

    accuracy                           1.00     85443
   macro avg       0.98      0.89      0.93     85443
weighted avg       1.00      1.00      1.00     85443



### Undersampling

In [15]:

from imblearn.under_sampling import NearMiss
from collections import Counter
ns = NearMiss(0.8)
feature_train_ns,target_train_ns = ns.fit_resample(features_train,target_train)
print("The Number of Classes before fit {}".format(Counter(target_train)))
print("The Number of Classes after fit {}".format(Counter(target_train_ns)))

C:\Users\rohit.pratapwar\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:591: FutureWarning: Pass sampling_strategy=0.8 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning,


The Number of Classes before fit Counter({0: 199017, 1: 347})
The Number of Classes after fit Counter({0: 433, 1: 347})


In [16]:
classifier = RandomForestClassifier()
classifier.fit(feature_train_ns,target_train_ns)

RandomForestClassifier()

In [17]:
pred = classifier.predict(features_test)
print(confusion_matrix(target_test,pred))
print(accuracy_score(target_test,pred))
print(classification_report(target_test,pred))

[[72568 12730]
 [    9   136]]
0.8509064522547195
              precision    recall  f1-score   support

           0       1.00      0.85      0.92     85298
           1       0.01      0.94      0.02       145

    accuracy                           0.85     85443
   macro avg       0.51      0.89      0.47     85443
weighted avg       1.00      0.85      0.92     85443



### Over Sampling

In [18]:
from imblearn.over_sampling import RandomOverSampler

In [19]:
os = RandomOverSampler(0.75)
feature_train_os,target_train_os = os.fit_resample(features_train,target_train)
print("The Number of Classes before fit {}".format(Counter(target_train)))
print("The Number of Classes after fit {}".format(Counter(target_train_os)))

C:\Users\rohit.pratapwar\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:591: FutureWarning: Pass sampling_strategy=0.75 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning,


The Number of Classes before fit Counter({0: 199017, 1: 347})
The Number of Classes after fit Counter({0: 199017, 1: 149262})


In [20]:
classifier = RandomForestClassifier()
classifier.fit(feature_train_os,target_train_os)

RandomForestClassifier()

In [21]:
pred = classifier.predict(features_test)
print(confusion_matrix(target_test,pred))
print(accuracy_score(target_test,pred))
print(classification_report(target_test,pred))

[[85294     4]
 [   29   116]]
0.9996137776061234
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85298
           1       0.97      0.80      0.88       145

    accuracy                           1.00     85443
   macro avg       0.98      0.90      0.94     85443
weighted avg       1.00      1.00      1.00     85443



In [22]:
from imblearn.ensemble import EasyEnsembleClassifier

In [23]:
easy = EasyEnsembleClassifier()
easy.fit(features_train,target_train)

EasyEnsembleClassifier()

In [24]:
pred = easy.predict(features_test)
print(confusion_matrix(target_test,pred))
print(accuracy_score(target_test,pred))
print(classification_report(target_test,pred))

[[82493  2805]
 [   10   135]]
0.9670540594314339
              precision    recall  f1-score   support

           0       1.00      0.97      0.98     85298
           1       0.05      0.93      0.09       145

    accuracy                           0.97     85443
   macro avg       0.52      0.95      0.54     85443
weighted avg       1.00      0.97      0.98     85443

